In [4]:
import pandas as pd
import numpy as np
import math
import requests
from pandas.io.json import json_normalize
import pandas_profiling

### Yelp API

To get more details (price range, rating, cuisine,...) on the establishments, we use the Yelp Fusion API and follow these two steps:


1. **Business Match** : The Business Match Endpoint lets us match business data from other sources against businesses on Yelp. We send data from Chicago Food Inspection (namely name and address of establishments) and get back the Yelp id of the establishment. 


2. **Business Details** : The Business Details Endpoint returns detailed business content based on their id (which we have obtained in the previous step). 

**API Keys**

Along with the parameters we send with each request, we need to send an API key with the headers. This allows Yelp to make sure that we do not exceed 5'000 requests per API Key. 
To get these, we simply need to visit the [Yelp Fusion](https://www.yelp.com/developers/v3/manage_app) and create a project. 


* irBVcEgoI6PeOStlUs77hlL-9r0MA5oa14yFIdtA_mtpe3ChIaN8rAyLcCGgQAFQdfI4cK2ZY84cx3ZFIMh2bDBgIotdi_9yN3Xlp23eE-9HF-Ou5vTrErRnngbIXXYx


* gWozwadRpB03ay6xxc0rrs4w5Qy5rzB59xQDaaKjcAeXmtZ6D_R8fNQCClj-rga4CgtDRMDqgYVkYn5KvXMn3MYXf5VKSttOE21qbxoCHebmOMb5k_9GFykHvVLZXXYx


* qslxiyR7VolDPWr5sOVBppQEYZqfxJpodCxt-iYXQWveW9n-2XFrHnPzmpsuew-jcLm2Ezi7twdDyodsdl8gP-c0BsY5QRewuODCv1NMZUhpKoISxKOHWeE0gW7ZXXYx


* ZvQ095-O2_NVNygRjjsFK8NhruSE-V51GV5oBUoEvGh52RnsRBHG_9CTx6aoZ1fpQgp8Si2WEZJv-flkhaIRuyKCw9g5OUStUTY9SNhO5BasXGa7KWhWeB80GIvaXXYx


* aiC1S6FQJ6Tg7DdBfRTBKgX0W8LTuli7hhLA_HF17nVFUFrZsjq501K6nPclqB_lWWWIniJ8JeCCHr1H273kyJK0DNTx4e5mzvjdkj7_5XqWOmXP7HOtOtFG5IjaXXYx


* GW1WdoxZIIGg92uc7-kL-rjR3B3q83B4r2NYEhUCXJPnDwPOAn5pOteAQcUDdZNBXFbzFpa8fw_bCr0zq9sF_hD-x7f1uW2PyRzso4JdkIZVOqV7uHxRhHRbxY_aXXYx


* K5PcgS12CfjM_SRDuDBpAbFkiMmJp0eHvBr6mVLtp7gFj-JjCUFgZtrS5OeUqL4GPlBKMPICNshHw1UAbhM5hnCZtX10Cs3sJlqqXU662TaExk2IyaVGk_IB2pDaXXYx


* Qkz_aqKy7I6YxaTOxpPOgh73GNXNw-eM8Pzjiel-Ffm6aGmrMnxrSbilTI0Ax7riPvNyyoQXvaauKS11-F5KZiyCcSOjFt-yCY9ckQwmLfiZC1C6P6bJvIThzRvVXXYx


* 4QbudQVuZzEVqmPkK2KcLs3HNF5WXXdP1iu1UKxwgC4FPLSSRfU-zaIPsrh8dm3lgAXXZa4S73snzkwrmqBeRuEgp7WWuts-OWvAng2TW8rVC-_EdCGRrLXjE5TaXXYx


* F9AJnjXDvkELW1vqPX-kpe2B8NaJ-5spLNgzODHl3XweQKRnPjuDfT2U6ZpUzcQcvKFT4em8PPLxF-liSStwmueVK1ha8gAFllnairC942ZvCBvDx52MttGk9ZPaXXYx


* kad0AuvpQdBV65JsgR6cz5TA_K7aFMfmp5oQSuDRDBHyTw7KOl60HHtg5ZLjmG80mwOM-8VBuUW9yy6vr6MTRGO7e_DlGiLAK1DfMvHUQWpH4AYhKUzb4wT2EpvaXXYx

In [26]:
API_KEY = 'kad0AuvpQdBV65JsgR6cz5TA_K7aFMfmp5oQSuDRDBHyTw7KOl60HHtg5ZLjmG80mwOM-8VBuUW9yy6vr6MTRGO7e_DlGiLAK1DfMvHUQWpH4AYhKUzb4wT2EpvaXXYx'
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

#### BUSINESS MATCH ENDPOINT: Make requests about establishment from the Chicago Food Inspections dataset

In [28]:
ENDPOINT = 'https://api.yelp.com/v3/businesses/matches'

#### Tests (test here for an individual establishment)

In [244]:
#Define the parameters
PARAM = {'name': 'LA CATRINA RESTAURANT LLC', 'address1': '3924 W DIVERSEY AVE ', 'city': 'Chicago', 'state': 'IL', 'country': 'US'}

response = requests.get(url=ENDPOINT, params=PARAM, headers = HEADERS)

#json_normalize(response.json()['businesses'])

In [247]:
response.headers

{'Server': 'nginx', 'Content-Type': 'application/json', 'x-routing-service': '10-65-208-15-useast1cprod; site=public_api_v3', 'x-b3-sampled': '0', 'x-zipkin-id': 'a463066aa41e7b6b', 'ratelimit-remaining': '0', 'ratelimit-resettime': '2019-11-24T00:00:00+00:00', 'ratelimit-dailylimit': '5000', 'x-proxied': '10-65-214-75-useast1cprod', 'Content-Length': '230', 'Accept-Ranges': 'bytes', 'Date': 'Sat, 23 Nov 2019 15:29:13 GMT', 'Via': '1.1 varnish', 'Connection': 'keep-alive', 'X-Served-By': 'cache-hhn4023-HHN', 'X-Cache': 'MISS', 'X-Cache-Hits': '0'}

#### Loading the inspections dataset
This dataframe contains all establishments from the Chicago Food Inspections

In [5]:
estab = pd.read_pickle("insp_businesses_pickle")

In [6]:
estab = estab[['DBA Name', 'AKA Name', 'Address']]
estab.head()

,DBA Name,AKA Name,Address
0,YOUNG SCHOLARS ACADEMY,YOUNG SCHOLARS ACADEMY,10926-10928 S WESTERN AVE
1,OSITO'S TAP,OSITO'S TAP,2553 S RIDGEWAY AVE
2,LAS SISTERS INC.,LAS SISTERS INC,2700 S TRUMBULL AVE
3,CARNICERIA Y FRUTERIA 'LOS ALTOS' INC.,CARNICERIA Y FRUTERIA LOS ALTOS,2959 W 40TH ST
4,FOODA- 150 N RIVERSIDE,FOODA 150 N RIVERSIDE,150 N RIVERSIDE PLZ


In [7]:
# to this dataframe, we add columns where we will record the Yelp answers
estab['response_id'] = None
estab['response_alias'] = None
estab['response_name'] =  None
estab['response_latitude'] =  None
estab['response_longitude'] =  None
estab['response_address1'] =  None

In [57]:
reamining_requests = response.headers['ratelimit-remaining']
print(f'remaining number of requests : {reamining_requests}')

remaining number of requests : 4399


#### Make requests for the first 5000 rows

In [30]:
def fill_df(dataframe, ENDPOINT):
    """
    Given a dataframe with restaurants
    and Given a list of responses
    Returns the same dataframe with additional columns coming from Yelp
    (condition : If there was 1 match with Yelp)
    And fills up the list of reponses with all responses
    """
    result = dataframe.copy()
    resps =  []
    for req_nb in range(len(result)):
        if(req_nb%100 == 0):
            print(f'req {req_nb} over {len(result)}')
        PARAM = {
        'name' : result.loc[req_nb]['DBA_Name'],
        'address1':result.loc[req_nb]['Address'],
        'city': 'Chicago',
        'state': 'IL',
        'country': 'US'}
        response = requests.get(url=ENDPOINT, params=PARAM, headers = HEADERS)
        resps.append(response)
        response_df = json_normalize(response.json()['businesses'])
        #if we get a match, we fill the dataframe 
        if (len(response_df) == 1):
            result['response_id'][req_nb] = response_df['id'][0]
            result['response_alias'][req_nb] = response_df['alias'][0]
            result['response_name'][req_nb] = response_df['name'][0]
            result['response_latitude'][req_nb] = response_df['coordinates.latitude'][0]
            result['response_longitude'][req_nb] = response_df['coordinates.longitude'][0]
            result['response_address1'][req_nb] = response_df['location.address1'][0]
    return result, resps

In [77]:
yelp_resp_df_0_5000 = estab[0:5000].copy()
yelp_resp_df_0_5000, yelp_resp_0_5000 = fill_df(yelp_resp_0_5000, ENDPOINT)

req 0 over 5000
req 100 over 5000
req 200 over 5000
req 300 over 5000
req 400 over 5000
req 500 over 5000
req 600 over 5000
req 700 over 5000
req 800 over 5000
req 900 over 5000
req 1000 over 5000
req 1100 over 5000
req 1200 over 5000
req 1300 over 5000
req 1400 over 5000
req 1500 over 5000
req 1600 over 5000
req 1700 over 5000
req 1800 over 5000
req 1900 over 5000
req 2000 over 5000
req 2100 over 5000
req 2200 over 5000
req 2300 over 5000
req 2400 over 5000
req 2500 over 5000
req 2600 over 5000
req 2700 over 5000
req 2800 over 5000
req 2900 over 5000
req 3000 over 5000
req 3100 over 5000
req 3200 over 5000
req 3300 over 5000
req 3400 over 5000
req 3500 over 5000
req 3600 over 5000
req 3700 over 5000
req 3800 over 5000
req 3900 over 5000
req 4000 over 5000
req 4100 over 5000
req 4200 over 5000
req 4300 over 5000
req 4400 over 5000
req 4500 over 5000
req 4600 over 5000
req 4700 over 5000
req 4800 over 5000
req 4900 over 5000


In [90]:
yelp_resp_df_0_5000.head()

,DBA Name,AKA Name,Address,response_id,response_alias,response_name,response_latitude,response_longitude,response_address1
0,YOUNG SCHOLARS ACADEMY,YOUNG SCHOLARS ACADEMY,10926-10928 S WESTERN AVE,None,None,None,None,None,None
1,OSITO'S TAP,OSITO'S TAP,2553 S RIDGEWAY AVE,None,None,None,None,None,None
2,LAS SISTERS INC.,LAS SISTERS INC,2700 S TRUMBULL AVE,None,None,None,None,None,None
3,CARNICERIA Y FRUTERIA 'LOS ALTOS' INC.,CARNICERIA Y FRUTERIA LOS ALTOS,2959 W 40TH ST,None,None,None,None,None,None
4,FOODA- 150 N RIVERSIDE,FOODA 150 N RIVERSIDE,150 N RIVERSIDE PLZ,None,None,None,None,None,None


In [31]:
yelp_resp_0_5000.isna().sum().apply(lambda x: '{}% missing values'.format(100 * x/len(yelp_resp_0_5000)))

DBA_Name                0.0% missing values
AKA_Name               0.64% missing values
Address                 0.0% missing values
response_id           30.72% missing values
response_alias        30.72% missing values
response_name         30.72% missing values
response_latitude     30.72% missing values
response_longitude    30.72% missing values
response_address1     30.72% missing values
dtype: object

In [264]:
#yelp_resp_df_0_5000.to_pickle('yelp_match_0_4999.pickle')

#### BUSINESS DETAILS ENDPOINT: extract price range, rating and cuisine
Now that we have made the match requests and obtained the establishments' ids, we can make more requests to get details. 

In [39]:
# we get rid of the unmatched establishments
matched_responses_5000 = yelp_resp_0_5000.dropna(subset=['response_id']).reset_index(drop=True)
#matched_responses_5000.head()

,DBA_Name,AKA_Name,Address,response_id,response_alias,response_name,response_latitude,response_longitude,response_address1
0,CARNICERIA Y FRUTERIA 'LOS ALTOS' INC.,CARNICERIA Y FRUTERIA LOS ALTOS,2959 W 40TH ST,jleGfLhZuxvRUdqAFstVGg,carniceria-y-fruteria-los-altos-chicago,Carniceria Y Fruteria Los Altos,41.8206,-87.6993,2959 W 40th St
1,CHICAGO'S FINEST BEEF AND BREW,CHICAGO'S FINEST BEEF AND BREW,234 S WABASH AVE,KwnuxV_YLkHUfP3dhe_qZg,als-beef-chicago-19,Al's Beef,41.8785,-87.6264,234 S Wabash Ave
2,PEKING MANDARIN RESTAURANT,PEKING MANDARIN RESTAURANT,3457-3459 W LAWRENCE AVE,4xQ7x-Td1nFshx7J0Bd6pw,peking-mandarin-chicago,Peking Mandarin,41.9682,-87.7151,3459 W Lawrence Ave
3,LA HUMITA,LA HUMITA,3466 N PULASKI RD,bh55xTHTQGLx95rPIj79ug,la-humita-chicago,La Humita,41.9447,-87.7277,3466 N Pulaski Rd
4,CHICAGO PRODUCE,CHICAGO PRODUCE,3500 W LAWRENCE AVE,WzXgyBi6K36UVEiluU7Rlg,chicago-produce-chicago,Chicago Produce,41.9686,-87.7162,3500 W Lawrence Ave
...,...,...,...,...,...,...,...,...,...
3459,FORNO ROSSO ON RANDOLPH,FORNO ROSSO ON RANDOLPH,1048 W RANDOLPH ST,2cDt9At556hf146QH_XkhQ,forno-rosso-pizzeria-napoletana-chicago-3,Forno Rosso Pizzeria Napoletana,41.8846,-87.6541,1048 W Randolph St
3460,CONGAS,CONGAS,7021 W HIGGINS AVE,qSrQDsG2BYdCmphu3-qrvA,congas-chicago,Congas,41.9802,-87.8033,7021 W Higgins Ave
3461,WRIGLEYVILLE ROOFTOPS,WRIGLEYVILLE ROOFTOPS,3637 N SHEFFIELD AVE,R4wLWK510K8VpYi_WJzX9g,wrigley-rooftops-chicago-9,Wrigley Rooftops,41.9478,-87.654,3609 N Sheffield
3462,CHEF XIONG,CHEF XIONG,2143 S ARCHER AVE,Ydn0Q4uqtokhlHTIwOHeTA,chef-xiong-taste-of-szechuan-chicago,Chef Xiong - Taste of Szechuan,41.8534,-87.6333,2143 S Archer Ave


#### Make requests for all 5000 establishments
We send the details requests and fill in the final dataframe

In [302]:
details_5000 = pd.DataFrame()
for req_nb in range(len(matched_responses_5000)):
    if (req_nb%100 == 0):
        print(f'Request {req_nb} over {len(matched_responses_5000)}')
    REQUEST_URL = 'https://api.yelp.com/v3/businesses/' + matched_responses_5000['response_id'][req_nb]
    response = requests.get(url=REQUEST_URL, headers = HEADERS)
    details_5000 = details_5000.append(json_normalize(response.json()))

Request 0 over len(matched_responses_5000)
Request 100 over len(matched_responses_5000)
Request 200 over len(matched_responses_5000)
Request 300 over len(matched_responses_5000)
Request 400 over len(matched_responses_5000)
Request 500 over len(matched_responses_5000)
Request 600 over len(matched_responses_5000)
Request 700 over len(matched_responses_5000)
Request 800 over len(matched_responses_5000)
Request 900 over len(matched_responses_5000)
Request 1000 over len(matched_responses_5000)
Request 1100 over len(matched_responses_5000)
Request 1200 over len(matched_responses_5000)
Request 1300 over len(matched_responses_5000)
Request 1400 over len(matched_responses_5000)
Request 1500 over len(matched_responses_5000)
Request 1600 over len(matched_responses_5000)
Request 1700 over len(matched_responses_5000)
Request 1800 over len(matched_responses_5000)
Request 1900 over len(matched_responses_5000)
Request 2000 over len(matched_responses_5000)
Request 2100 over len(matched_responses_5000)


In [305]:
len(details_5000)

0

In [36]:
cleaned = pd.read_pickle('./datasets/cleaned_inspections.pickle')

In [37]:
cleaned = cleaned.drop_duplicates(subset=['DBA Name', 'AKA Name', 'Address'])[['DBA Name', 'AKA Name', 'Address']].reset_index(drop=True)

In [38]:
cleaned.isna().sum().apply(lambda x: '{}% missing values'.format(100 * x/len(cleaned)))

DBA Name    0.0% missing values
AKA Name    0.0% missing values
Address     0.0% missing values
dtype: object

In [39]:
cleaned = cleaned.rename(columns={"DBA Name": "DBA_Name"})
cleaned['response_id'] = None
cleaned['response_alias'] = None
cleaned['response_name'] =  None
cleaned['response_latitude'] =  None
cleaned['response_longitude'] =  None
cleaned['response_address1'] =  None

In [40]:
yelp_resp_df_0_300 = cleaned[0:300].copy()
yelp_resp_df_0_300, yelp_resp_0_300 = fill_df(yelp_resp_df_0_300, ENDPOINT)

req 0 over 300
req 100 over 300
req 200 over 300
